In [70]:
import json
import pandas as pd
import os
import random
import itertools

SENTIMENT_DATA_DIR = "../data/sentiment_explanation"

In [67]:
import os, json

combined_data = []
for file in os.listdir("../data/sentiment_explanation/"):
    if "process_id" in file:
        filename = os.path.join("../data/sentiment_explanation/", file)
        data = json.load( open(filename, "r")  )

        combined_data += data
len(combined_data)

old_data = json.load(open("../data/sentiment_explanation/train_with_summaries_with_explanations.json", "r"))
json.dump(old_data + combined_data, fp=open("../data/sentiment_explanation/train_with_summaries_with_explanations.json", "w"),indent=4, default=set_default)

3

In [5]:
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError


In [71]:
explanation_data = json.load( open(os.path.join(SENTIMENT_DATA_DIR, "train_with_summaries_with_explanations.json"), "r")  )


In [105]:
plutchik = ["anger", "fear", "sadness", "disgust", "surprise", "anticipation", "trust", "joy"]
cleaned_explanation_data = []

for i in explanation_data:
    new_dict = dict(i)
    if "[CHOICE]" in new_dict["choice"]:
        new_dict["choice"] = new_dict["choice"].split("[CHOICE]:")[-1].strip().lower()
    elif "[choices]" in new_dict["choice"]:
        new_dict["choice"] = new_dict["choice"].split("[choices]:")[-1].strip().lower()
    elif "[CHOICES]" in new_dict["choice"]:
        new_dict["choice"] = new_dict["choice"].split("[CHOICES]:")[-1].strip().lower()

    if new_dict["choice"] in plutchik:
        del new_dict["raw_response"], new_dict["choices"], new_dict["text"]
        new_dict["text"] = new_dict["summary"]
        del new_dict["summary"]
        cleaned_explanation_data.append(new_dict)

In [109]:
json.dump(cleaned_explanation_data, fp=open("../data/sentiment_explanation/train_formatted.json", "w"),indent=4, default=set_default)

In [98]:
plutchik = ["anger", "fear", "sadness", "disgust", "surprise", "anticipation", "trust", "joy"]

In [182]:
new_explanation_data = json.load( open(os.path.join(SENTIMENT_DATA_DIR, "train_formatted.json"), "r")  )

emotions = []
for i in new_explanation_data:
    emotions.append(i["choice"])
import plotly.express as px

fig = px.histogram(emotions)
fig.show()

In [188]:
new_explanation_data = json.load( open(os.path.join(SENTIMENT_DATA_DIR, "train_formatted.json"), "r")  )

In [192]:
example

{'text': 'The team celebrated their hard-fought victory, jubilantly hoisting the championship trophy into the air as confetti rained down upon them, marking their triumphant season.',
 'choice': 'joy',
 'explanation': "The exuberant celebration of the team's victory demonstrates a clear sense of joy and elation, highlighting the fulfillment and satisfaction of their hard work and dedication.",
 'id': 'generated'}

In [193]:
for example in new_explanation_data:
    if "id" not in example:
        example["id"] = "generated"
    if example["id"] != "generated": 
        continue
    if "generated_raw" in example:
        del example["generated_raw"]
    example["text"] = example["text"].replace("[TEXT]: ", "").strip()

In [197]:
new_explanation_data[0]

{'id': '1198',
 'choice': 'trust',
 'explanation': ' The text conveys a sense of growing confidence in the sewage testing method, indicating trust in its effectiveness.',
 'text': 'Epidemiologists found that testing sewage viral loads accurately predicts COVID-19 cases and hospitalizations, providing early warnings. The method is increasingly trusted.'}

In [198]:
cleaned_explanation_data = []

fear_count = 0

for i in new_explanation_data:
    if i["choice"] != "fear":
        cleaned_explanation_data.append(i)
    else:
        if fear_count <= 200:
            cleaned_explanation_data.append(i)
            fear_count += 1
        else:
            continue

In [199]:
emotions = []
for i in cleaned_explanation_data:
    emotions.append(i["choice"])
import plotly.express as px

fig = px.histogram(emotions)
fig.show()

In [ ]:
json.dump(cleaned_explanation_data, fp=open("../data/sentiment_explanation/train_cleaned.json", "w"),indent=4, default=set_default)

In [33]:
sentiment_data = json.load( open(os.path.join(SENTIMENT_DATA_DIR, "train.json"), "r")  )

summary_data = {}

for id in sentiment_data:
    example = dict()
    text = sentiment_data[id]["Reddit Post"]
    choices = [i["Emotion"] for i in list(itertools.chain.from_iterable(list((sentiment_data[id]["Annotations"]).values())))]

    example["text"] = text
    example["choices"] = choices

    summary_data[id] = example

In [34]:
summary_data["0"]

{'text': 'Im suppose to get my first vaccination soon. Everyone else in my household and my mother is fully vaccinated. After doing a lot of research to questions I had regarding the vaccines I felt more comfortable finally booking to get one. It seemed like they are/were working. Lockdowns were being lifted, I saw online people doing things social and doctors celebrating etc. I avoid the news about covid for my mental health. Pretty much what I find out about the pandemic pops up on my feed. Now Im seeing so much conflicting information at the same time and Im confused and defeated. UK says pfizer vaccine is 88% effective against delta variant but is that only against hospitalizations? What about actually getting it and suffering? Israel says pfizer went from 64% to 39% . This may not be helpful but Ive recently read about a lot of people on reddit who are fully vaccinated getting covid - and what concerns me more is a lot of them are saying its kicking their ass. Why are they making 

In [35]:
# Define prompt handler
import openai
import random


class PromptHandler:
    def __init__(self, api_key, prompt_template):
        """ """
        openai.api_key = random.choice(api_key)
        self.prompt_template = prompt_template
        self.model = prompt_template["model"]

    def generate_summary(self, text):
        """
        """
        prompt = ""
        prompt += self.prompt_template["instruction"] + "\n"
        prompt += text

        summary = openai.ChatCompletion.create(model=self.model,messages=[{"role": "system", "content": ""},
                                                                            {"role": "user", "content": prompt},],
                                                                            temperature=0,
                                                                            top_p=1,
                                                                            frequency_penalty=0,
                                                                            presence_penalty=0,).choices[0].message.content

        return summary
    
    def generate_explanation(self, example):
        """
        """
        prompt = ""
        prompt += self.prompt_template["instruction"] + "\n"
        prompt += "[TEXT]: " + example["text"] + "\n"
        prompt += "[CHOICES]: " + ", ".join(example["choices"]) + "\n"
        prompt += "[CHOICE]: \n + [EXPLANATION]: "

        choice, explanation = openai.ChatCompletion.create(model=self.model,messages=[{"role": "system", "content": ""},
                                                                            {"role": "user", "content": prompt},],
                                                                            temperature=0,
                                                                            top_p=1,
                                                                            frequency_penalty=0,
                                                                            presence_penalty=0,).choices[0].message.content.split("[EXPLANATION]:")

        return choice, explanation

In [36]:
keys = json.load(open("../prompting/openai_keys.json", "r"))

prompt_template = json.load(open("../prompting/sentiment_explanation/generate_summaries.json", "r"))
model = PromptHandler(keys, prompt_template)

In [38]:
output = model.generate_summary(summary_data["0"]["text"])

In [39]:
output

"I'm about to get my first vaccination, feeling more comfortable after research, but now confused and defeated by conflicting information about vaccine effectiveness and breakthrough cases."

In [175]:
import os, json

combined_data = []
for file in os.listdir("../data/sentiment_explanation/"):
    if "process_id" in file:
        filename = os.path.join("../data/sentiment_explanation/", file)
        data = json.load( open(filename, "r")  )
        new_data = []

        for example in data:
            new_example = {}
            new_text = example["text"].split("[TEXT]:")[-1]
            new_example["text"] = new_text.strip()
            new_example["choice"] = example["choice"]
            new_example["explanation"] = example["explanation"].strip()
            new_data.append(new_example)

        combined_data += new_data

In [176]:
len(combined_data)

100

In [170]:
json.dump(combined_data, fp=open("check.json", "w"),indent=4, default=set_default)

In [128]:
old_data = json.load(open("../data/sentiment_explanation/train_formatted.json", "r"))
json.dump(old_data + combined_data, fp=open("../data/sentiment_explanation/train_formatted.json", "w"),indent=4, default=set_default)

In [133]:
generated_data = json.load(open("check.json", "r"))

In [148]:
set(generated_data[2]["text"].lower().split()).intersection(set(generated_data[65]["text"].lower().split()))

{'a',
 'as',
 'environment',
 'grass.',
 'group',
 'i',
 'littering',
 'of',
 'park,',
 'people',
 'sight',
 'such',
 'the',
 'their',
 'through',
 'walked'}

In [162]:
from nltk.corpus import stopwords

stopword_list = stopwords.words("english")

In [180]:
extracted_data = list(combined_data)

for index, example in enumerate(combined_data):
    for other_index, other_example in enumerate(combined_data):
        if index != other_index:
            text_1 = [i for i in example["text"].lower().split() if i not in stopword_list]
            text_2 = [i for i in other_example["text"].lower().split() if i not in stopword_list]

            print(text_1)
            print(text_2)
            input()

            if len(set(text_1).intersection(set(text_2))) > 5:
                try:
                    extracted_data.remove(other_example)
                except ValueError:
                    continue

['entire', 'family', 'gathered', 'together', 'weekend', 'retreat', 'mountains.', 'laughter,', 'storytelling,', 'bonding', 'moments', 'created', 'cherished', 'memories', 'last', 'lifetime.']
['new', 'vr', 'headset', 'arrived', 'today!', "can't", 'wait', 'immerse', 'virtual', 'worlds', 'explore', 'new', 'experiences.']
['entire', 'family', 'gathered', 'together', 'weekend', 'retreat', 'mountains.', 'laughter,', 'storytelling,', 'bonding', 'moments', 'created', 'cherished', 'memories', 'last', 'lifetime.']
['got', 'latest', 'smartphone', 'model', 'features', 'amazing!', 'camera', 'quality', 'outstanding,', 'new', 'apps', 'much', 'fun', 'use.']


KeyboardInterrupt: Interrupted by user

In [181]:
json.dump(extracted_data, fp=open("extracted_data.json", "w"),indent=4, default=set_default)

In [178]:
len(combined_data), len(extracted_data)

(100, 61)